## Model usage
model creation steps are hidden due to company intellectual property

In [3]:
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import numpy as np
import pandas as pd

In [6]:
class Model(nn.Module):
    def __init__(self,
                 input_dim,
                 embedding_dim,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):

        super(Model, self).__init__()

        self.word_embeddings = nn.Embedding(input_dim, embedding_dim, padding_idx=1)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional = bidirectional)
        self.hidden2tag = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)

    def forward(self, x):
        embeds = self.word_embeddings(x)
        lstm_out, _ = self.lstm(embeds)
        tag_space = self.hidden2tag(lstm_out)

        return tag_space

In [14]:
INPUT_DIM = 81 # max of formula + 1
EMBEDDING_DIM = 128
HIDDEN_DIM = 64
OUTPUT_DIM = 11 # 5 possible points
N_LAYERS = 2
BIDIRECTIONAL = False
DROPOUT = 0.25
PAD_IDX = 1

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Model(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT).to(device)
dict_state = torch.load('../models/complexity-prediction-model/best.pt')
model.load_state_dict(dict_state)
model.eval()

Model(
  (word_embeddings): Embedding(81, 128, padding_idx=1)
  (lstm): LSTM(128, 64)
  (hidden2tag): Linear(in_features=64, out_features=11, bias=True)
)

In [17]:
def predict(
        model,
        text,
        length
):
    predictions = []
    with torch.no_grad():
        model.eval()  # evaluation mode

        # forward pass and loss calculation
        outputs = model(text)


        _, predicted = torch.max(outputs.data, 1)
        predictions += predicted[:length].detach().cpu().tolist()

    return predictions

In [18]:
prev = [3, 4]
max_size = 50
next_complexity = 20
prev.append(next_complexity)

prev_tensor  = torch.Tensor(prev).long()
complexity_padding = torch.Tensor([0] * (max_size - len(prev))).long()
to_predict = torch.concat((prev_tensor, complexity_padding))

l = predict(model, to_predict, len(prev))
l

[5, 6, 7]